In [ ]:
import os
import json
import time
import random
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from cerebras.cloud.sdk import Cerebras

# --- НАСТРОЙКИ ---
# ==============================================================================
# Эти параметры позволяют легко настраивать поведение скрипта.

# 1. Ваши API ключи Cerebras
# Замените на реальные ключи! Получить их можно здесь: https://cloud.cerebras.ai/platform/org_.../apikeys
# Рекомендуется использовать переменные окружения для безопасности, например:
# export CEREBRAS_API_KEY_1="cbkey_xxxxxxxxxxxxxxxxxxxxxxxxxxxx"
API_KEYS = []

# 2. Настройки модели-учителя Cerebras
MODEL_NAME_TEACHER = "qwen-3-235b-a22b"

# 3. Параметры параллелизма
MAX_WORKERS = 4 # Количество одновременных запросов к API. Зависит от лимитов API и вашей пропускной способности.

# 4. Настройки повторных попыток при ошибках API
MAX_RETRIES_PER_KEY = 3 # Максимальное количество попыток с одним ключом до его смены
RETRY_DELAY = 5         # Пауза в секундах между повторными попытками

# 5. Параметры генерации датасета
NUM_UNIQUE_QUESTIONS_TO_GENERATE = 400 # Общее количество уникальных "тем" вопросов для генерации.
QUESTIONS_BATCH_SIZE = 3               # Количество вопросов, генерируемых за один API-запрос на первом этапе.
# Сколько различных стратегий рассуждения мы хотим получить для КАЖДОГО уникального вопроса.
# Если установлено 3, и у нас 3 стратегии (CoT, PoT, SkipThinking), то для каждого вопроса будет по 1 примеру каждой стратегии.
NUM_REASONING_STRATEGIES_PER_QUESTION = 6 

# 6. Параметры рассуждений
WORDS_IN_REASONING = "100-200" # Ожидаемый объем "мыслей" (chain_of_thought) в словах для CoT/PoT.
WORDS_IN_CHUNK = "30-50"       # Ожидаемый объем слов в каждом "блоке мысли" для Skip-Thinking.

# 7. Файл вывода
OUTPUT_DATASET_FILE = "qwen3_adaptive_reasoning_dataset.jsonl"

# --- ПРОМПТЫ ---
# ==============================================================================
# Здесь определены промпты для различных этапов и типов рассуждений.

# Промпт для генерации начальных вопросов
QUESTION_GENERATOR_PROMPT = """
Твоя задача — придумать {count} уникальных и сложных вопросов на русском языке, требующих многошаговых рассуждений для ответа.
Вопросы должны быть из следующей области: {topic}.
Если это применимо к данной теме, сгенерируй несколько вопросов, которые повторяют одну и ту же базовую идею, но используют **разные числа, условия, контекст или параметры**, чтобы каждый вопрос оставался уникальным, но проверял схожий тип рассуждений.

Верни ответ ТОЛЬКО в виде валидного JSON-массива строк. Не добавляй никаких других пояснений или текста до и после JSON.

Пример:
["Каким образом можно оценить примерный вес облака?", "Если бы у тени был вес, что бы повлияло на его изменение в течение дня?", "Если в саду 10 яблонь, и каждая дает по 20 кг яблок, сколько всего яблок будет собрано?", "Если в саду 15 яблонь, и каждая дает по 15 кг яблок, сколько всего яблок будет собрано?"]

Твой JSON-массив с {count} вопросами:
"""

# Промпт для генерации Chain-of-Thought (CoT) рассуждений
REASONING_SOLVER_PROMPT_COT = """
Ты — экспертная система, способная к глубоким пошаговым рассуждениям. Твоя цель — решить поставленную задачу, подробно объяснив свой мыслительный процесс, используя только текстовые рассуждения (Chain-of-Thought).
Ты должен сгенерировать ответ в формате JSON.

ФОРМАТ ВЫВОДА:
Ответ должен быть валидным JSON-объектом со следующими ключами: "question", "chain_of_thought", "answer", "thought_type".

1.  **`chain_of_thought`**: Здесь ты должен продемонстрировать свой мыслительный процесс. Структурируй его логически, шаг за шагом. Начни с анализа вопроса, затем опиши план решения и последовательно изложи свои рассуждения. Весь текст должен быть на русском языке. Целевая длина — примерно {WORDS_IN_REASONING} слов.
2.  **`answer`**: Здесь укажи ТОЛЬКО конечный, четкий и сжатый ответ без лишних пояснений.
3.  **`thought_type`**: Всегда устанавливай значение "CoT" для этого типа рассуждения.

Вопрос для решения:
{question}

Твой JSON-ответ:
"""

# Промпт для генерации Program-of-Thought (PoT) рассуждений
REASONING_SOLVER_PROMPT_POT = """
Ты — экспертная система, способная к глубоким пошаговым рассуждениям. Твоя цель — решить поставленную задачу, подробно объяснив свой мыслительный процесс, используя программный код (Python) для промежуточных вычислений и проверки.
Ты должен сгенерировать ответ в формате JSON.

ФОРМАТ ВЫВОДА:
Ответ должен быть валидным JSON-объектом со следующими ключами: "question", "chain_of_thought", "answer", "thought_type".

1.  **`chain_of_thought`**: Здесь ты должен продемонстрировать свой мыслительный процесс. Структурируй его логически, шаг за шагом. Начни с анализа вопроса, затем опиши план решения, затем последовательно изложи свои рассуждения, **включая блоки кода Python для выполнения вычислений**. Используй Markdown для выделения кода. Весь текст должен быть на русском языке. Целевая длина — примерно {WORDS_IN_REASONING} слов.
2.  **`answer`**: Здесь укажи ТОЛЬКО конечный, четкий и сжатый ответ без лишних пояснений.
3.  **`thought_type`**: Всегда устанавливай значение "PoT" для этого типа рассуждения.

Пример `chain_of_thought` для задачи: "Сколько будет 5 * 8 + 3?"
"Для решения этой задачи я сначала выполню умножение, а затем сложение.
```python
result = 5 * 8
```
Полученный результат умножения равен 40. Теперь добавлю 3:
```python
final_result = 40 + 3
```
Итоговый результат - 43."

Вопрос для решения:
{question}

Твой JSON-ответ:
"""

# Промпт для генерации Skip-Thinking рассуждений (с блоками)
REASONING_SOLVER_PROMPT_SKIP_THINKING = """
Ты — экспертная система, способная к глубоким пошаговым рассуждениям. Твоя цель — решить поставленную задачу, подробно объяснив свой мыслительный процесс, разбивая его на логические, атомарные "блоки мыслей".
Каждый блок должен быть заключен в специальные теги: <chunk_start> и <chunk_end>.
В конце каждого блока, после <chunk_end>, ты также должен указать один из следующих маркеров:
- <continue_thinking>: если для решения задачи требуется дальнейшее рассуждение.
- <end_of_thought>: если ты считаешь, что после этого блока рассуждений можно сразу перейти к финальному ответу.

Ты должен сгенерировать ответ в формате JSON.

ФОРМАТ ВЫВОДА:
Ответ должен быть валидным JSON-объектом со следующими ключами: "question", "chain_of_thought", "answer", "thought_type".

1.  **`chain_of_thought`**: Здесь ты должен продемонстрировать свой мыслительный процесс, разделенный на блоки. Пример:
    <chunk_start>
    [Шаг 1 рассуждения. Длина примерно {WORDS_IN_CHUNK} слов.]
    </chunk_end><continue_thinking>
    <chunk_start>
    [Шаг 2 рассуждения. Длина примерно {WORDS_IN_CHUNK} слов.]
    </chunk_end><end_of_thought>
    Или, для простой задачи:
    <chunk_start>
    [Единственный блок рассуждения. Длина примерно {WORDS_IN_CHUNK} слов.]
    </chunk_end><end_of_thought>
    Весь текст должен быть на русском языке.
2.  **`answer`**: Здесь укажи ТОЛЬКО конечный, четкий и сжатый ответ без лишних пояснений.
3.  **`thought_type`**: Всегда устанавливай значение "SkipThinking" для этого типа рассуждения.

Вопрос для решения:
{question}

Твой JSON-ответ:
"""

# Список всех доступных стратегий рассуждения и их промптов
REASONING_STRATEGIES = {
    "CoT": REASONING_SOLVER_PROMPT_COT,
    "PoT": REASONING_SOLVER_PROMPT_POT,
    "SkipThinking": REASONING_SOLVER_PROMPT_SKIP_THINKING,
}

# Темы для генерации вопросов
QUESTION_TOPICS = [
    "логические загадки", "простая математика", "бытовые задачи на логику",
    "задачи на пространственное мышление", "физические парадоксы", "задачи на оценку",
    "алгоритмические задачи", "экологические задачи", "исторические гипотезы",
    "экономические сценарии", "задачи на вероятность", "научные концепции",
    "геометрические задачи", "финансовые расчеты", "задачи на время",
    "криптография (основы)", "анализ данных (концепции)", "биологические процессы"
]

# --- ГЛОБАЛЬНЫЕ ПЕРЕМЕННЫЕ И УПРАВЛЕНИЕ КЛИЕНТОМ ---
# ==============================================================================

current_key_index = 0
cerebras_client = None

def get_client():
    """Инициализирует или обновляет клиент Cerebras с текущим API-ключом."""
    global cerebras_client
    if not API_KEYS or not API_KEYS[0].strip():
        raise ValueError("API ключи Cerebras не заданы в списке API_KEYS. Пожалуйста, добавьте их.")
    
    api_key = API_KEYS[current_key_index]
    if not api_key.strip() or "cbkey_xxxxxxxx" in api_key: # Проверка на пустой или плейсхолдер
        raise ValueError(f"API ключ Cerebras #{current_key_index + 1} недействителен или не задан. Пожалуйста, замените его.")
    
    cerebras_client = Cerebras(api_key=api_key)
    return cerebras_client

def switch_to_next_key():
    """Переключается на следующий API-ключ в списке, если доступны."""
    global current_key_index
    current_key_index += 1
    if current_key_index < len(API_KEYS):
        tqdm.write(f"⚠️ [INFO] Лимит для ключа исчерпан или произошла ошибка. Переключаюсь на ключ #{current_key_index + 1}.")
        get_client() # Инициализируем клиент с новым ключом
        return True
    else:
        tqdm.write("❌ [ERROR] Все API ключи Cerebras исчерпаны. Невозможно продолжить.")
        return False
        
def safe_json_parse(response_content: str) -> dict | list | None:
    """
    Безопасно извлекает JSON из строки, даже если он окружен другим текстом.
    Ищет первый '{' или '[' и последний '}' или ']'.
    """
    try:
        first_brace_idx = response_content.find('{')
        first_bracket_idx = response_content.find('[')
        
        start_pos = -1
        if first_brace_idx != -1 and first_bracket_idx != -1:
            start_pos = min(first_brace_idx, first_bracket_idx)
        elif first_brace_idx != -1:
            start_pos = first_brace_idx
        elif first_bracket_idx != -1:
            start_pos = first_bracket_idx
        else:
            return None # JSON не найден

        if start_pos == -1: return None 

        end_char = '}' if response_content[start_pos] == '{' else ']'
        end_pos = response_content.rfind(end_char)
        
        if end_pos == -1 or end_pos < start_pos: return None 
        
        json_str = response_content[start_pos : end_pos + 1]
        return json.loads(json_str)
        
    except (json.JSONDecodeError, IndexError) as e:
        tqdm.write(f"‼️ [WARN] Не удалось декодировать JSON. Ошибка: {e}\nОтвет модели (начало): {response_content[:200]}...")
        return None

# --- ЯДРО ЛОГИКИ: ВЫПОЛНЕНИЕ ЗАПРОСОВ ---
# ==============================================================================

def execute_api_call(prompt: str, task_description: str, model_name: str):
    """
    Выполняет API-запрос к Cerebras с автопереключением ключей и повторными попытками.
    """
    global cerebras_client
    
    current_attempt_with_key = 0
    while current_key_index < len(API_KEYS):
        if current_attempt_with_key >= MAX_RETRIES_PER_KEY:
            if not switch_to_next_key():
                return None 
            current_attempt_with_key = 0 
        
        try:
            chat_completion = cerebras_client.chat.completions.create(
                messages=[{"role": "user", "content": prompt}],
                model=model_name,
                temperature=0.7, 
                max_tokens=2048, 
            )
            return chat_completion.choices[0].message.content.strip()
            
        except Exception as e:
            current_attempt_with_key += 1
            tqdm.write(f"‼️ [WARN] Ошибка '{task_description}' (ключ #{current_key_index + 1}, попытка {current_attempt_with_key}/{MAX_RETRIES_PER_KEY}): {e}")
            time.sleep(RETRY_DELAY) 
        
    return None 

def generate_questions_batch(count: int, topic: str) -> list[str]:
    """Генерирует партию уникальных вопросов по заданной теме."""
    prompt = QUESTION_GENERATOR_PROMPT.format(count=count, topic=topic)
    response_content = execute_api_call(prompt, f"Генерация вопросов по теме '{topic}'", MODEL_NAME_TEACHER)
    
    if not response_content:
        return []
    
    questions = safe_json_parse(response_content)
    return [q.strip() for q in questions if isinstance(q, str) and q.strip()] if isinstance(questions, list) else []

def generate_reasoning_example(question: str, strategy_type: str, prompt_template: str) -> dict | None:
    """
    Генерирует рассуждения и ответ для одного вопроса, используя заданную стратегию.
    Включает метаданные thought_type для дистилляции Adaptive Thinking.
    """
    prompt_format_args = {
        "question": question,
        "WORDS_IN_REASONING": WORDS_IN_REASONING
    }
    if strategy_type == "SkipThinking":
        prompt_format_args["WORDS_IN_CHUNK"] = WORDS_IN_CHUNK

    prompt = prompt_template.format(**prompt_format_args)
    response_content = execute_api_call(prompt, f"Генерация {strategy_type} для вопроса: '{question[:50]}...'", MODEL_NAME_TEACHER)

    if not response_content:
        return None
        
    data_from_model = safe_json_parse(response_content)
    
    required_keys = ["question", "chain_of_thought", "answer", "thought_type"]
    if not isinstance(data_from_model, dict) or not all(key in data_from_model for key in required_keys):
        tqdm.write(f"‼️ [WARN] В ответе модели отсутствуют необходимые ключи или некорректный формат для {strategy_type}. Пропускаю. Ответ: {response_content[:100]}...")
        return None

    data_from_model['thought_type'] = data_from_model.get('thought_type', strategy_type)

    assistant_content = (
        f"<thought_type>{data_from_model['thought_type']}</thought_type>\n"
        f"<think>\n{data_from_model['chain_of_thought']}\n</think>\n" 
        f"{data_from_model['answer']}"
    )
    
    return {
        "messages": [
            {"role": "user", "content": data_from_model["question"]},
            {"role": "assistant", "content": assistant_content}
        ]
    }

# --- ОСНОВНОЙ КОД ---
# ==============================================================================

def main():
    """
    Основная функция, координирующая процесс генерации датасета
    для дистилляции Adaptive Thinking и Skip-Thinking.
    """
    try:
        get_client() # Инициализация первого клиента
    except ValueError as e:
        print(f"❌ [ОШИБКА] {e}")
        return

    # --- ЭТАП 1: Генерация уникальных вопросов ---
    print("\n" + "="*80)
    print(f"🚀 ЭТАП 1: Генерация {NUM_UNIQUE_QUESTIONS_TO_GENERATE} уникальных тем вопросов с помощью Cerebras...")
    all_unique_questions = set() 
    
    with tqdm(total=NUM_UNIQUE_QUESTIONS_TO_GENERATE, desc="Генерация уникальных тем вопросов") as pbar:
        while len(all_unique_questions) < NUM_UNIQUE_QUESTIONS_TO_GENERATE and current_key_index < len(API_KEYS):
            needed = NUM_UNIQUE_QUESTIONS_TO_GENERATE - len(all_unique_questions)
            batch_size = min(needed, QUESTIONS_BATCH_SIZE)
            
            new_questions_batch = generate_questions_batch(batch_size, random.choice(QUESTION_TOPICS))
            
            if not new_questions_batch:
                if current_key_index >= len(API_KEYS) - 1: # Проверяем, если это последний ключ
                    tqdm.write("❌ [ERROR] Не удалось сгенерировать вопросы, и все ключи исчерпаны. Остановка.")
                    break
                # Если просто не было возвращено вопросов, но ключи еще есть, делаем паузу и продолжаем
                time.sleep(RETRY_DELAY) 
                continue

            prev_count = len(all_unique_questions)
            all_unique_questions.update(new_questions_batch) 
            pbar.update(len(all_unique_questions) - prev_count) 
            time.sleep(0.5) 

    unique_questions_list = list(all_unique_questions)
    print(f"✅ Сгенерировано {len(unique_questions_list)} уникальных тем вопросов.")

    if not unique_questions_list:
        print("Вопросы не были сгенерированы. Завершение работы.")
        return

    # --- ЭТАП 2: Подготовка задач для параллельной генерации различных стратегий рассуждения ---
    print("\n" + "="*80)
    print(f"🧠 ЭТАП 2: Подготовка задач для генерации рассуждений различных типов...")
    
    tasks_for_reasoning = []
    strategy_types_list = list(REASONING_STRATEGIES.keys())
    
    for question in unique_questions_list:
        for i in range(NUM_REASONING_STRATEGIES_PER_QUESTION):
            # Случайный выбор стратегии. Можно задать веса, если нужно больше определенных типов.
            # Например, SkipThinking и CoT могут быть более приоритетными.
            chosen_strategy_type = random.choices(
                strategy_types_list, 
                weights=[0.2, 0.5, 0.3] if "SkipThinking" in strategy_types_list else [0.5, 0.5], # Пример весов
                k=1
            )[0]
            chosen_prompt_template = REASONING_STRATEGIES[chosen_strategy_type]
            
            tasks_for_reasoning.append({
                "question": question,
                "strategy_type": chosen_strategy_type,
                "prompt_template": chosen_prompt_template
            })
    random.shuffle(tasks_for_reasoning) 

    print(f"� Сформировано {len(tasks_for_reasoning)} задач на генерацию рассуждений.")
    
    # --- ЭТАП 3: Параллельная генерация рассуждений и ответов ---
    print("\n" + "="*80)
    print(f"🚀 ЭТАП 3: Генерация рассуждений (различных типов) в {MAX_WORKERS} потоков...")
    
    final_samples = []
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        future_to_task = {
            executor.submit(generate_reasoning_example, t['question'], t['strategy_type'], t['prompt_template']): t
            for t in tasks_for_reasoning
        }
        
        progress_bar = tqdm(as_completed(future_to_task), total=len(tasks_for_reasoning), desc="Генерация рассуждений и ответов")
        
        for future in progress_bar:
            original_task = future_to_task[future]
            try:
                result_data = future.result()
                if result_data:
                    final_samples.append(result_data)
            except Exception as e:
                tqdm.write(f"❌ [ERROR] Ошибка в потоке при обработке задачи для вопроса '{original_task['question'][:50]}...' ({original_task['strategy_type']}). Ошибка: {e}")

    # --- ЭТАП 4: Запись сгенерированных примеров в файл ---
    print("\n" + "="*80)
    print(f"💾 ЭТАП 4: Запись {len(final_samples)} сгенерированных примеров в файл '{OUTPUT_DATASET_FILE}'...")
    
    with open(OUTPUT_DATASET_FILE, 'a', encoding='utf-8') as f:
        for entry in final_samples:
            f.write(json.dumps(entry, ensure_ascii=False) + '\n')

    # --- Завершение ---
    print("\n" + "="*80)
    print("🎉 Генерация завершена!")
    print(f"Датасет сохранен в файл: {OUTPUT_DATASET_FILE}")
    print(f"Всего сгенерировано {len(final_samples)} валидных примеров для дистилляции Adaptive Thinking.")
    if final_samples:
        print("\nПример структуры одной записи в файле:")
        print(json.dumps(final_samples[0], indent=2, ensure_ascii=False)) 
    print("="*80)


if __name__ == "__main__":
    main()

Гистограмма длины sample-ов по токенам

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
import logging
import os

# Настройка логирования
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# --- НАСТРОЙКИ ---
# ==============================================================================
# Имя файла датасета, сгенерированного предыдущим скриптом
OUTPUT_DATASET_FILE = "D:\Work\Python\Parser_for_distil_dataset\qwen3_adaptive_reasoning_dataset.jsonl"
# Имя файла для сохранения гистограммы
HISTOGRAM_OUTPUT_FILE = "token_length_histogram.png"

# --- ФУНКЦИЯ ТОКЕНИЗАЦИИ ---
# ==============================================================================
# Попытка импортировать tiktoken для более точного подсчета токенов.
# Если tiktoken не установлен, используется простой подсчет слов.
try:
    import tiktoken
    ENCODER = tiktoken.get_encoding("cl100k_base")
    def tokenize_text(text: str) -> int:
        """Подсчитывает количество токенов в тексте с помощью tiktoken."""
        return len(ENCODER.encode(text))
    logger.info("Используется tiktoken для токенизации.")
except ImportError:
    logger.warning("Библиотека 'tiktoken' не найдена. Используется простой подсчет слов для оценки токенов.")
    def tokenize_text(text: str) -> int:
        """Подсчитывает количество слов в тексте как прокси для токенов."""
        return len(text.split())
    
# --- ОСНОВНАЯ ЛОГИКА ---
# ==============================================================================

def plot_token_length_histogram():
    """
    Генерирует гистограмму распределения длины токенов для сгенерированных примеров.
    Читает данные из OUTPUT_DATASET_FILE, подсчитывает токены и сохраняет гистограмму.
    """
    token_lengths = []

    # Проверяем существование файла датасета
    if not os.path.exists(OUTPUT_DATASET_FILE):
        logger.error(f"Файл датасета '{OUTPUT_DATASET_FILE}' не найден. Пожалуйста, убедитесь, что он существует и был сгенерирован.")
        return

    logger.info(f"Загрузка данных из '{OUTPUT_DATASET_FILE}' для анализа длины токенов...")
    try:
        with open(OUTPUT_DATASET_FILE, 'r', encoding='utf-8') as f:
            for line in f:
                try:
                    entry = json.loads(line)
                    # Ожидаем структуру {"messages": [{"role": "user", "content": "..."}, {"role": "assistant", "content": "..."}]}
                    if "messages" in entry and isinstance(entry["messages"], list) and len(entry["messages"]) == 2:
                        user_content = entry["messages"][0].get("content", "")
                        assistant_content = entry["messages"][1].get("content", "")
                        
                        # Объединяем пользовательский и ассистентский контент для подсчета общей длины
                        full_text = user_content + " " + assistant_content
                        token_count = tokenize_text(full_text)
                        token_lengths.append(token_count)
                    else:
                        logger.warning(f"Неожиданный формат записи в файле: {line.strip()}. Пропускаю.")
                except json.JSONDecodeError as e:
                    logger.warning(f"Ошибка декодирования JSON в строке: {line.strip()}. Пропускаю. Ошибка: {e}")
                except Exception as e:
                    logger.warning(f"Неожиданная ошибка при обработке строки: {line.strip()}. Пропускаю. Ошибка: {e}")
    except Exception as e:
        logger.critical(f"Ошибка при чтении файла '{OUTPUT_DATASET_FILE}': {e}")
        return

    if not token_lengths:
        logger.warning("Не удалось извлечь длины токенов. Возможно, файл пуст или имеет неверный формат после фильтрации.")
        return

    logger.info(f"Собрано {len(token_lengths)} длин токенов. Создание гистограммы...")

    # Создание гистограммы
    plt.figure(figsize=(12, 7)) # Устанавливаем размер фигуры для лучшей читаемости
    
    num_bins = min(50, int(np.sqrt(len(token_lengths)))) if len(token_lengths) > 0 else 10
    
    plt.hist(token_lengths, bins=num_bins, edgecolor='black', alpha=0.7, color='skyblue')
    
    # Добавляем заголовок и подписи осей
    plt.title('Распределение длины примеров в токенах', fontsize=16)
    plt.xlabel('Длина в токенах', fontsize=12)
    plt.ylabel('Количество примеров', fontsize=12)
    
    # Добавляем сетку для лучшей читаемости
    plt.grid(axis='y', alpha=0.75)
    
    # Добавляем линии для среднего и медианы
    mean_length = np.mean(token_lengths)
    median_length = np.median(token_lengths)
    plt.axvline(mean_length, color='r', linestyle='dashed', linewidth=1.5, label=f'Среднее: {mean_length:.2f}')
    plt.axvline(median_length, color='g', linestyle='dashed', linewidth=1.5, label=f'Медиана: {median_length:.2f}')
    plt.legend() # Отображаем легенду для линий среднего и медианы

    # Оптимизируем расположение элементов на графике
    plt.tight_layout()
    
    # Сохраняем гистограмму в файл
    plt.savefig(HISTOGRAM_OUTPUT_FILE)
    logger.info(f"Гистограмма сохранена в '{HISTOGRAM_OUTPUT_FILE}'")
    logger.info("Процесс анализа длины токенов завершен.")

if __name__ == "__main__":
    plot_token_length_histogram()


Выгрузка датасета на Hugging Face

In [ ]:
from huggingface_hub import login

login(token="Ваш токен")

In [ ]:
from datasets import load_dataset

# Путь к файлу, созданному вашим скриптом
dataset_file_path = "qwen3_adaptive_reasoning_dataset.jsonl"

# Загрузка JSONL файла
my_dataset = load_dataset('json', data_files=dataset_file_path, split="train")

# Укажите свой username на Hugging Face и желаемое имя для датасета
repo_id = "ваш_username/qwen3_adaptive_reasoning_dataset"

# Загрузка на Hugging Face Hub
my_dataset.push_to_hub(repo_id)

print(f"Датасет успешно загружен на Hugging Face Hub по адресу: https://huggingface.co/datasets/{repo_id}")